# Prétraitement des données

## Suppression des éléments indésirables de nos tweets

In [2]:
# Imports nécessaires
import pandas as pd
import numpy as np
import string 
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from nltk.stem import WordNetLemmatizer

In [11]:
# Lecture et stockage de la base de données
tweet_df = pd.read_pickle('../../delphes/data/final4_clean.csv')
tweet_df.head()
new_test_df = pd.read_pickle('../../delphes/data/extended_tweet_df')

In [12]:
age = []
sexe = []

In [13]:
y = np.arange(0, tweet_df.shape[0])
tweet_df['index'] = y
tweet_df.set_index('index', inplace=True)

In [7]:
new_test_df = pd.DataFrame(columns=['mep_id', 'name', 'country', 'group', 'nat_group', 'twitter', 'content'])

In [8]:
%%time
k = 0
for j in range(tweet_df.shape[0]):
    for i in range(len(tweet_df['content'][j])):
        new_test_df.loc[k] = tweet_df.loc[j,'mep_id':'twitter']
        new_test_df.loc[k, 'content'] = tweet_df['content'][j][i]
        k += 1

KeyboardInterrupt: 

In [14]:
# Remove the undesirable elements in the entire dataframe
def rmurl_df(df, column_name):
    '''
    This function removes all the URLs, the #hashtag and the @user of a column made of strings.
    Be careful to apply it BEFORE all the other preprocessing steps (if not it wont'
    be recognized as a URL)
    '''
    df = df.copy()
    df[column_name] = df[column_name].str.replace('http\S+|www.\S+|@\S+|#\S+', '', case=False)
    return df



In [15]:
# Lowercase the tweet's column
def lower_df(df, column_name):
    '''
    This function lowercases a column made of strings.
    '''
    df = df.copy()
    df[column_name] = df[column_name].str.lower()
    return df

In [16]:
# Remove the numbers in the tweet's column
def rmnumbers_df(df, column_name):
    '''
    This function removes all the digits of a column made of strings.
    '''
    df = df.copy()
    def remove_numbers(text):
        return ''.join(word for word in text if not word.isdigit())
    df[column_name] = df[column_name].apply(remove_numbers)
    return df

In [17]:
# Remove the undesirable punctuations in the tweet's column
def rmpunct_df(df, column_name):
    '''
    This function removes all the punctuations, all the "rt" and remove multiple spaces
    of a column made of strings.
    '''
    punct = string.punctuation
    df = df.copy()
    def replace_punct(text):
        for punctu in punct:
            text = text.replace(punctu, ' ')
        text = text.replace(' rt ','')
        text = " ".join(text.split())
        return text
    df[column_name] = df[column_name].apply(replace_punct)
    return df

In [18]:
# Remove the stopwords in the tweet's column
def rmstopwords_df(df, column_name):
    '''
    This function removes all the stopwords of a column made of strings.
    '''
    df = df.copy()
    stop_words = stopwords.words('english')
    def remove_stopwords(text):
        for word in stop_words:
            text = text.replace(f' {word} ', ' ')
        return text
    df[column_name] = df[column_name].apply(remove_stopwords)
    return df

In [19]:
# Lemmatize a column in a dataset
def lemmatize_df(df, column_name):
    '''
    This function lemmatize the words of a column made of strings.
    '''
    df = df.copy()
    def lemmatize(text):
        lemmatizer = WordNetLemmatizer()
        retour = []
        for word in text:
            retour.append(lemmatizer.lemmatize(word))
        text = ''.join(word for word in retour)
        return text

    df[column_name] = df[column_name].apply(lemmatize)
    return df

In [20]:
# Erase all the words that are 1-letter or 2-letters long
def erase_fewletter_df(df, column_name):
    '''
    One or two letters words are deleted from the dataset.
    '''
    df = df.copy()
    def tester(text):
        text = ' '.join( [w for w in text.split() if len(w)>2] )
        return text

    df[column_name] = df[column_name].apply(tester)
    return df


In [21]:
# Remove the undesirable emojis in the entire dataframe
def rmemojis_df(df):
    '''
    This function removes all the emojis of a column made of strings.
    Be careful to translate in latin alphabet before applying this function : 
    it also removes cyrillic alphabet
    '''
    df = df.copy()
    df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
    return df

In [22]:
clean_df = rmurl_df(new_test_df, 'content')
clean_df = lower_df(clean_df, 'content')
clean_df = rmnumbers_df(clean_df, 'content')
clean_df = rmpunct_df(clean_df, 'content')
clean_df = rmstopwords_df(clean_df, 'content')
clean_df = lemmatize_df(clean_df, 'content')
clean_df = erase_fewletter_df(clean_df, 'content')
clean_df = rmemojis_df(clean_df)

In [23]:
clean_df[clean_df['country'] == 'Ireland'].head()

,mep_id,name,country,group,nat_group,twitter,content
24722,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,the northern ireland protocol must protected l...
24723,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,member special committee beating cancer look f...
24724,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,this month shining light childrens cancer parl...
24725,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,yesterday told europe need keep speed amp infr...
24726,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,this letter sent taoiseach clear seeking clari...


In [24]:
clean_df['content'][24722]

'the northern ireland protocol must protected line real effort must made earnest week bridge gap talks come table many times good faith amp view securing future partnership respects standards union'

In [25]:
testbase_df = clean_df[clean_df['country'] == 'Ireland']
testbase_deputy_df = tweet_df[tweet_df['country'] == 'Ireland']

## Création du modèle

### Traitement des données d'entraînement et de test

In [26]:
# Cette fonction retourne automatiquement X_train, X_test, y_train, y_test de notre base de données twitter.
def get_train_test_objects(df):
    '''
    Les étapes que cette fonction réalise sont en commentaires.
    '''
    # Copie de la base de données pour éviter les problèmes d'assignation abusive.
    df = df.copy() 
    # Récupération de tous les tweets et du nom du député qui les a posté. Création de la cible y.
    df = df[['name', 'content']]
    y = pd.get_dummies(df['name'])
    # Transformation des tweets en suite de mots (strings) dans une liste.
    sentences = df['content']
    sentences_inter = []
    for sentence in sentences:
        sentences_inter.append(sentence.split())
    # Séparation des données d'entraînement et de test
    sentences_train, sentences_test, y_train, y_test = train_test_split(sentences_inter, y, test_size = 0.3)
    # Vectorisation des phrases
    word2vec = Word2Vec(sentences=sentences_train)
    # Création des données d'entrée.
    X_train = embedding(word2vec,sentences_train)
    X_test = embedding(word2vec,sentences_test)
    X_train_pad = pad_sequences(X_train, padding='post',value=-1000, dtype='float32')
    X_test_pad = pad_sequences(X_test, padding='post',value=-1000, dtype='float32')
    # Création des données cibles.
    y_train = y_train.values
    y_test = y_test.values
    # Sorties de la fonction
    return X_train_pad, y_train, X_test_pad, y_test

In [27]:
def embed_sentence(word2vec, sentence):
    y = []
    for word in sentence:
        if word in word2vec.wv.vocab.keys():
           y.append(word2vec[word])
    return np.array(y)

def embedding(word2vec, sentences):
    
    y = []
    for sentence in sentences:
        y.append(embed_sentence(word2vec, sentence))
    return y

In [28]:
X_train, y_train, X_test, y_test = get_train_test_objects(testbase_df)

c:\users\psmag\.venvs\delphes\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


### Initiation du modèle

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

def init_model():

    model = Sequential()
    model.add(layers.Masking(mask_value = -1000))
    model.add(layers.LSTM(13, activation='tanh'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])
    return model


### Entraînement du modèle et évaluation

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience = 100, monitor='val_loss', restore_best_weights=True)
model = init_model()
model.fit(X_train, y_train, batch_size = 8, epochs=1000, validation_split = 0.3, callbacks = [es])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1470 samples, validate on 630 samples
Epoch 1/1000
1470/1470 [==============================] - 5s 3ms/sample - loss: 2.3021 - acc: 0.1061 - val_loss: 2.2971 - val_acc: 0.1079
Epoch 2/1000
1470/1470 [==============================] - 3s 2ms/sample - loss: 2.2926 - acc: 0.1177 - val_loss: 2.2814 - val_acc: 0.1413
Epoch 3/1000
1470/1470 [==============================] - 3s 2ms/sample - loss: 2.2666 - acc: 0.1503 - val_loss: 2.2527 - val_acc: 0.1381
Epoch 4/1000
1470/1470 [==============================] - 4s 2ms/sample - loss: 2.2366 - acc: 0.1633 - val_loss: 2.2394 - val_acc: 0.1381
Epoch 5/1000
1470/1470 [==============================] - 4s 2ms/sample - loss: 2.2211 - acc: 0.1796 - val_loss: 2.2407 - val_acc: 0.1333
Epoch 6/1000
1470/1470 [==============================] - 4s 2ms/sample - loss: 2.2115 -

Epoch 55/1000
1470/1470 [==============================] - 3s 2ms/sample - loss: 2.1464 - acc: 0.2150 - val_loss: 2.1971 - val_acc: 0.1984
Epoch 56/1000
1470/1470 [==============================] - 3s 2ms/sample - loss: 2.1462 - acc: 0.2170 - val_loss: 2.1993 - val_acc: 0.1937
Epoch 57/1000
1470/1470 [==============================] - 3s 2ms/sample - loss: 2.1476 - acc: 0.2293 - val_loss: 2.2084 - val_acc: 0.1857
Epoch 58/1000
1470/1470 [==============================] - 3s 2ms/sample - loss: 2.1473 - acc: 0.2259 - val_loss: 2.2042 - val_acc: 0.2000
Epoch 59/1000
1470/1470 [==============================] - 3s 2ms/sample - loss: 2.1504 - acc: 0.2102 - val_loss: 2.2041 - val_acc: 0.2048
Epoch 60/1000
1470/1470 [==============================] - 3s 2ms/sample - loss: 2.1399 - acc: 0.2327 - val_loss: 2.1985 - val_acc: 0.1921
Epoch 61/1000
1470/1470 [==============================] - 3s 2ms/sample - loss: 2.1458 - acc: 0.2204 - val_loss: 2.2042 - val_acc: 0.1984
Epoch 62/1000
1470/1470 [==

Epoch 114/1000
1470/1470 [==============================] - 3s 2ms/sample - loss: 2.0887 - acc: 0.2483 - val_loss: 2.2357 - val_acc: 0.1905
Epoch 115/1000
1470/1470 [==============================] - 3s 2ms/sample - loss: 2.0799 - acc: 0.2544 - val_loss: 2.2421 - val_acc: 0.1825
Epoch 116/1000
1470/1470 [==============================] - 4s 3ms/sample - loss: 2.0855 - acc: 0.2571 - val_loss: 2.2251 - val_acc: 0.1778
Epoch 117/1000
1470/1470 [==============================] - 4s 3ms/sample - loss: 2.0944 - acc: 0.2415 - val_loss: 2.2523 - val_acc: 0.1810
Epoch 118/1000
1470/1470 [==============================] - 3s 2ms/sample - loss: 2.0895 - acc: 0.2483 - val_loss: 2.2424 - val_acc: 0.1889
Epoch 119/1000
1470/1470 [==============================] - 3s 2ms/sample - loss: 2.0812 - acc: 0.2510 - val_loss: 2.2298 - val_acc: 0.1921
Epoch 120/1000
1470/1470 [==============================] - 3s 2ms/sample - loss: 2.0782 - acc: 0.2531 - val_loss: 2.2547 - val_acc: 0.2032
Epoch 121/1000
1470/

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
# Renvoie le député le plus proche de votre tweet
def predict_deputy(df, model, tweet, by_tweet = False):
    '''
    La fonction prend la base de données originale (par député), un modèle entraîné et un texte en entrée.
    Elle renvoie le député le plus proche du texte proposé.
    Attention : le texte en entrée doit être une liste d'au moins deux éléments (strings).
    Quand by_tweet = False, on ressort le député le plus proche de l'ENSEMBLE des tweets.
    Quand by_tweet = True, on sort le député le plus proche POUR CHAQUE tweet.
    '''
    tweet_inter = []
    for tw in tweet:
        tweet_inter.append(tw.split())
    X_example = embedding(word2vec,tweet_inter)
    X_example_pad = pad_sequences(X_example, padding='post',value=-1000, dtype='float32')
    prediction = model.predict(X_example_pad)
    if not by_tweet:
        deputy = list(df['name'])[prediction.sum(axis=0).argmax()]
        return deputy
    else:
        deputies_by_tweet = []
        for element in prediction:
            deputies_by_tweet.append(list(df['name'])[element.argmax()])
        return deputies_by_tweet

In [ ]:
prediction = predict_deputy(testbase_deputy_df,
                            model, 
                            ['the northern ireland protocol must protected line real effort must made earnest week bridge gap talks come table many times good faith amp view securing future partnership respects standards union', 
                             'member special committee beating cancer look forward working members contributing important fight cancer',
                             'this month shining light childrens cancer parliament buildings lit gold highlight survivors childrens cancer well diagnosed cancer'], 
                            by_tweet=False)

In [ ]:
prediction

In [567]:
testbase_df['content'][24724]

'this month shining light childrens cancer parliament buildings lit gold highlight survivors childrens cancer well diagnosed cancer'

In [560]:
testbase_df

,mep_id,name,country,group,nat_group,twitter,content
24722,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,the northern ireland protocol must protected l...
24723,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,member special committee beating cancer look f...
24724,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,this month shining light childrens cancer parl...
24725,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,yesterday told europe need keep speed amp infr...
24726,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,this letter sent taoiseach clear seeking clari...
...,...,...,...,...,...,...,...
130490,197863,Maria WALSH,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,mariawalsheu,brendan its favourite item desk
130491,197863,Maria WALSH,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,mariawalsheu,monday mornings mean tidy desk chaos ahead wee...
130492,197863,Maria WALSH,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,mariawalsheu,fergal post kick new week thank folks like mak...
130493,197863,Maria WALSH,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,mariawalsheu,it took rare vision unique leadership see beyo...
